# Imports

In [ ]:
from Functions import make_experiment, estimate_staining_thresholds, test_skeletonization, extract_features, threshold_change, plot_thresholds

# User Input

### General Settings

Provide information regarding directories, used barcodes, and define the file name for results.
Setup .xls file must be in source folder. See GitHub readme for more information.

In [ ]:
# Point to experiment folder. Has to include setup .xls file.
source = "/Your/Absolute/Path"

# Analysis directory. If not present will automatically create 4 folder substructure for standardization in subsequent notebooks.
analysis_dir = "/Your/Absolute/Path"

# Point to individual folders relative of the "source" folder for extracted organoid files from individual imaged microscopy plates. See GitHub readme for more information.
folder =["Relative/Path1", "Relative/Path2"]

# List of used Barcodes, unique for imaged microscopy plates. List must have the same length and order as "folder" structure. Barcodes must be the same as in .xls setup file name. See GitHub readme for more information.
barcodes = ["201013MK001EB", "201013MK002EB"]

result_file_name = "1_FE_Results" # Filename for results file. Will be augmented by date and time of saving.

### Measurement settings

In [ ]:
# Antibody information. Will be used to name features in resulting .csv and AnnData file.
## Syntax: "Name_AntibodyMix" : ["Staining Ch1", "Staining Ch2", "Staining Ch3", "Staining Ch4"]
stainings = {
            "AB1": ["DAPI", "pSRC", "MUC2", "FABP1"],
            "AB2": ["DAPI", "AQP5", "CLDN2", "FABP1"],
            }

quantiles_to_calc = [0.25, 0.50, 0.75, 0.99] # List of quantiled to be calculated during feature extraction for every staining.

pixel_spacing = 0.325 # Pixel spacing in µm

step_distance = 50 # Distance in µm of slicing subsections in organoid crypts.

# Load and display experimental setup
Loads information from .xls setup files to subsequently link them to features of extracted organoids. Will display setup of all found imaging plates, with each well showing information regarding its conditions in the format of [Medium condition, used antibody mix, used cell line, additional condition]

In [ ]:
experiment_setup = make_experiment(source, barcodes)

# Estimate thresholds

### Automatic estimation
Estimates thresholds in time course experiments automatically by picking 0 random organoids per timepoint (stored in "Other" in the .xls setup file) within the control_condition for all stainings. Subsequently runs a triangle-segmentation on each picked organoid and returns the quantile (q) of this list. Thresholds can be changed by hand subsequently if not fitting.

In [ ]:
thresholds, dict_org, timepoints_lst = estimate_staining_thresholds(source = source, 
                                                                    folder = folder, 
                                                                    stainings = stainings, 
                                                                    experiment_setup = experiment_setup, 
                                                                    control_condition = "None", # Must be key in Medium tab of .xls setup file. Set to "None" if not applicable. 
                                                                    n = 20, # Number of organodis picked randomly per timepoint
                                                                    seed = 0, # Random seed
                                                                    sigma = 1, # Sigma for gaussian blurring
                                                                    q = 0.5 # Quantile used to pick final threshold
                                                                   )

### Manual change
Optional manual change of non-fitting automatic thresholds.

In [ ]:
thresholds = threshold_change(threshold = thresholds,
                              staining = "DAPI", # Staining to change
                              new_threshold = 800 # Integer of new threshold
                             )

### Test thresholds
Plots the distribution of thresholds found per stainings, we well as the currently used threshold in an kdeplot. Additionally, depicts one random organoid per staining and timepoint with the set threshold as a minimum value.

In [ ]:
fig = plot_thresholds(thresholds = thresholds, 
                      dict_org = dict_org, 
                      timepoints_lst = timepoints_lst)

# Test of crypt features extraction

Used to test parameters important to extract crypt features, such as sigm_skeleton, n_angle_determination, and radius_multiplier. Set parameters will be used in a subsequent step for feature extraction. Will show found crypts as well as an approximate for the non-crpyt region marked by a circle on n random organoids per plate.

In [ ]:
sigma_skeleton = 3 # Integer. Sigma for gaussian blurring of mask before skeletonization. Sharp segmentation edges may lead to difficulties, necessating higher sigma_skeleton numbers.
n_angle_determination = 50 # Integer. Number of pixel used at the end of each branch to estimate its direction until the segmentation border. If the number is higher than the length of the branch, the whole branch will be used instead.
radius_multiplier = 0.5 # Float. Multiplier of maximum-inscribed-circle radius to estimate crypt-free region. The lower the number, the more sensititve the algorithm is to smaller crypts.

test_skeletonization(folder, 
                     source, 
                     barcodes, 
                     pixel_spacing, 
                     n = 10, # Integer. Number of randomly chosen organoids per imaging plate.
                     seed = 0, # Integer. Random seed. 
                     sigma_skeleton = sigma_skeleton, 
                     n_angle_determination = n_angle_determination, 
                     radius_multiplier = radius_multiplier
                    )

# Extract Features

Function extracts morphological and staining features from all segmented organoids within the source+folder structure. Results will be saved in an .csv and .h5ad file in the analysis_dir+"2_Results"+result_file_name. .h5ad output of this function should be used as an input for the subsequent notebook "2_Filtering". 

In [ ]:
extract_features(source = source,
                 folder = folder,
                 analysis_dir = analysis_dir,
                 barcodes = barcodes,
                 experiment_setup = experiment_setup,
                 thresholds = thresholds,
                 stainings = stainings,
                 result_file_name = result_file_name,
                 radius_multiplier = radius_multiplier,
                 pixel_spacing = pixel_spacing,
                 sigma_skeleton = sigma_skeleton,
                 quantiles_to_calc = quantiles_to_calc)